In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import shutil
import keras
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D,Reshape, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, UpSampling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * (y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1-y_pred), 0, 1)))
    

    f1_val = tp / ( tp + ( (1/2) * (fp+fn) ) + K.epsilon())
    return f1_val

In [4]:
from glob import glob
trn='D:/data/PennA/Penn_Action/*/'
tr= glob(trn)

len(tr)



15

In [5]:
train = []
val = []
test = []
train_y = []
val_y = []
test_y = []

y = 0
for i in tr:
    
    #print(i)
    x = glob(i+'/*/')
    
    #shuffle(x)
    t,tt = train_test_split( x , test_size=0.1, random_state=42)
    t, vv = train_test_split( t , test_size=0.1, random_state=42)
    
    for j in t:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
        
        train.append(j)
        train_y.append(y)
    
    for j in vv:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
            
        val.append(j)
        val_y.append(y)
        
    for j in tt:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
            
        test.append(j)
        test_y.append(y)
        
    y = y+1

from tensorflow.keras.utils import to_categorical
tra_y =  np.array(to_categorical(train_y))
va_y  =  np.array(to_categorical(val_y))
te_y  =  np.array(to_categorical(test_y))

(train, tra_y) = shuffle(train, tra_y)
(val, va_y) = shuffle(val, va_y)
(test, te_y) = shuffle(test, te_y)

In [20]:

def get_te(k , a) :
    x = glob(k+'/*')
    imgdata=[]
    for i in range(0,5):
        
        a = Image.open(x[i])
        b = a.resize((224, 224))
        c = np.array(b)
        imgdata.append(c.reshape(224,224,3))
        
    idata = np.array(imgdata)
    X_train = idata
    X_train = X_train.astype('float32') / 255.
    #print(np.shape(X_train))
    return X_train

def get_cat(k) :
    return np.array(k)

In [21]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, filename , labels, batch_size) :
    self.filename = filename
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    y_train = get_cat(batch_y)
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x]), np.array( y_train )

In [23]:
class My_Test_Generator(keras.utils.Sequence) :
  
  def __init__(self, filename , batch_size) :
    self.filename = filename
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x])

In [24]:
batch_size = 16

my_training_batch_generator = My_Custom_Generator(train, tra_y, batch_size)
my_validation_batch_generator = My_Custom_Generator(val, va_y, batch_size)

In [25]:
arr = np.array(My_Test_Generator(test, batch_size).__getitem__(1))
for i in range(2,len(x)):
    x = My_Test_Generator(test, batch_size).__getitem__(i)
    arr = np.concatenate((arr,x),axis=0)

In [26]:
np.shape(arr)

(16, 5, 224, 224, 3)

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense

num_frames = 5
frame_height = 224
frame_width = 224
num_channels = 3
num_classes = 15  # Change this to the number of classes in your dataset

model = Sequential()

# CNN
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu'), input_shape=(num_frames, frame_height, frame_width, num_channels)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(8, (3, 3), activation='relu')))
model.add(TimeDistributed(Flatten()))

# LSTM
model.add(LSTM(50))

# Fully connected layer
model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_84 (TimeDi  (None, 5, 222, 222, 128)  3584     
 stributed)                                                      
                                                                 
 time_distributed_85 (TimeDi  (None, 5, 111, 111, 128)  0        
 stributed)                                                      
                                                                 
 time_distributed_86 (TimeDi  (None, 5, 109, 109, 128)  147584   
 stributed)                                                      
                                                                 
 time_distributed_87 (TimeDi  (None, 5, 54, 54, 128)   0         
 stributed)                                                      
                                                                 
 time_distributed_88 (TimeDi  (None, 5, 52, 52, 64)   

In [36]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.9,
    staircase=True)
optimizer = keras.optimizers.Adam(lr_schedule)

In [ ]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics='accuracy')
model.fit_generator(generator=my_training_batch_generator, epochs = 100,validation_data = my_validation_batch_generator)

C:\Users\shaif\AppData\Local\Temp\ipykernel_1528\789722107.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=my_training_batch_generator, epochs = 100,validation_data = my_validation_batch_generator)
C:\Users\shaif\AppData\Local\Temp\ipykernel_1528\2168406815.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/100
118/118 [==============================] - 83s 665ms/step - loss: 2.6860 - accuracy: 0.0768 - val_loss: 2.6619 - val_accuracy: 0.0986
Epoch 2/100
118/118 [==============================] - 37s 314ms/step - loss: 2.6365 - accuracy: 0.1115 - val_loss: 2.4510 - val_accuracy: 0.1737
Epoch 3/100
118/118 [==============================] - 38s 321ms/step - loss: 2.5004 - accuracy: 0.1724 - val_loss: 2.4328 - val_accuracy: 0.1643
Epoch 4/100
118/118 [==============================] - 38s 320ms/step - loss: 2.3572 - accuracy: 0.2263 - val_loss: 2.2658 - val_accuracy: 0.2347
Epoch 5/100
118/118 [==============================] - 40s 336ms/step - loss: 2.2476 - accuracy: 0.2561 - val_loss: 2.3179 - val_accuracy: 0.2113
Epoch 6/100
118/118 [==============================] - 40s 340ms/step - loss: 2.1883 - accuracy: 0.2716 - val_loss: 2.2176 - val_accuracy: 0.2488
Epoch 7/100
118/118 [==============================] - 44s 371ms/step - loss: 2.0921 - accuracy: 0.3058 - val_loss: 2.2175 -

In [ ]:
predictions = model.predict_generator(My_Test_Generator(test, batch_size), verbose=0)
from numpy import array
from numpy import argmax
from sklearn.preprocessing import Ofrom sklearn.metrics import accuracy_score
score = accuracy_score(test_y, p)
scoreneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
p = np.argmax(predictions, axis=1)
from sklearn.metrics import accuracy_score
score = accuracy_score(test_y, p)
score

In [ ]:
lossAndMetrics = model.evaluate(arr, te_y)
lossAndMetrics

In [13]:
#Patch Encoder with Conv2D ,  LSTM , Pos_Emd

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection =keras.Sequential(
            [
                (layers.Conv2D(4, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                (layers.Conv2D(8, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                TimeDistributed(Flatten()),
                layers.Dense(projection_dim)
            ]) 
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded 

In [14]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models

inputs = layers.Input(shape= (10,40,40,3) )

encoded_patches = (PatchEncoder(10, 32 )) (inputs)

for _ in range(6):
    
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    attention_output = layers.MultiHeadAttention (  num_heads=6, key_dim=32, dropout=0.1 )  (x1, x1)
    
    lstm_output =  LSTM(32,return_sequences=True,dropout=0.3)(x1)

    x2 = layers.Add()([attention_output, encoded_patches,lstm_output])

        # Layer Normalization and MLP
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)


    x3 = keras.Sequential(layers.Dense(units=32, activation=tf.nn.gelu) )(x3)

        # Skip connection
    encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

representation = layers.GlobalAvgPool1D()(representation)

outputs = layers.Dense(units=15, activation="softmax") ( representation)

model = keras.Model(inputs=inputs, outputs=outputs)
#model.summary()

In [15]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics='accuracy')
model.fit_generator(generator=my_training_batch_generator, epochs = 50,validation_data = my_validation_batch_generator)

C:\Users\chowd\AppData\Local\Temp\ipykernel_788\1958852391.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=my_training_batch_generator, epochs = 50,validation_data = my_validation_batch_generator)
C:\Users\chowd\AppData\Local\Temp\ipykernel_788\2168406815.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/50
59/59 [==============================] - 80s 1s/step - loss: 2.7010 - accuracy: 0.0975 - val_loss: 2.5393 - val_accuracy: 0.1495
Epoch 2/50
59/59 [==============================] - 75s 1s/step - loss: 2.4694 - accuracy: 0.1551 - val_loss: 2.3966 - val_accuracy: 0.1916
Epoch 3/50
59/59 [==============================] - 73s 1s/step - loss: 2.3188 - accuracy: 0.2116 - val_loss: 2.2126 - val_accuracy: 0.2383
Epoch 4/50
59/59 [==============================] - 73s 1s/step - loss: 2.1786 - accuracy: 0.2425 - val_loss: 2.0953 - val_accuracy: 0.2850
Epoch 5/50
59/59 [==============================] - 73s 1s/step - loss: 2.0074 - accuracy: 0.3177 - val_loss: 2.0630 - val_accuracy: 0.3131
Epoch 6/50
59/59 [==============================] - 74s 1s/step - loss: 1.8937 - accuracy: 0.3609 - val_loss: 2.0963 - val_accuracy: 0.3551
Epoch 7/50
59/59 [==============================] - 74s 1s/step - loss: 1.7027 - accuracy: 0.4270 - val_loss: 2.1092 - val_accuracy: 0.3598
Epoch 8/50
59/59 [==